### Assignment-01

#### 1. 第一节课的作业如下：

1. 复现课程的代码， BFS 路线搜索与文本自动生成；
2. 回答以下问题，并将问题发送至deeplearning.nlp.zh@gmail.com中：
```
    2.1. what do you want to acquire in this course？
    2.2. what problems do you want to solve？
    2.3. what’s the advantages you have to finish you goal?
    2.4. what’s the disadvantages you need to onvercome to finish you goal?
    2.5. How will you plan to study in this course period?
```
3. 完成以下联系，进一步体会 Data Driving的编程思想，完成你的第一个较为复杂的对话机器人。

#### 2. 如何提交

1. 代码 + 此 jupyter 相关，提交至自己的 github 中；
2. 第2问，整理发送至deeplearning.nlp.zh@gmail.com邮箱。

#### 3. 截止时间

此次作业截止时间为 2019.4.6日



## 基于模式匹配的对话机器人实现

### Pattern Match

机器能否实现对话，这个长久以来是衡量机器人是否具有智能的一个重要标志。 Alan Turing早在其文中就提出过一个测试机器智能程度的方法，该方法主要是考察人类是否能够通过对话内容区分对方是机器人还是真正的人类，如果人类无法区分，我们就称之为具有”智能“。而这个测试，后来被大家叫做”图灵测试“，之后也被翻拍成了一步著名电影，叫做《模拟游戏》。 



既然图灵当年以此作为机器是否具备智能的标志，这项任务肯定是复杂的。自从 1960s 开始，诸多科学家就希望从各个方面来解决这个问题，直到如今，都只能解决一部分问题。 目前对话机器人的建立方法有很多，今天的作业中，我们为大家提供一共快速的基于模板的对话机器人配置方式。

此次作业首先希望大家能够读懂这段程序的代码，其次，在此基于我们提供的代码，**能够把它改造成汉语版本，实现对话效果。**

```
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)
```

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成  "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [1]:
# 定义占位符
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

# .startwith('str', start_index, end_index) 
# The method .isalpha() checks whether the string consists of alphabetic characters only.
# all(iterable) returns True if all elements in an iterable are true.
# any(iterable) returns True if at least one element of an iterable is true.

In [2]:
# 判断 saying 和已有话语模板的结构是否一致
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True 
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:]) 
        
# 递归：
# Base case -- 字符串第一部分如果不是占位符，若要满足匹配一定相同;
# Operation -- 匹配剩余部分

### 例如

In [3]:
pat_match('I want ?X'.split(), "I want holiday".split())

True

In [4]:
pat_match('I have dreamed a ?X'.split(), "I dreamed about dog".split())

False

In [5]:
pat_match('I dreamed about ?X'.split(), "I dreamed about dog".split())

True

### 获得匹配的变量

以上的函数能够判断两个 pattern 是不是相符，但是我们更加希望的是获得每个variable对应的是什么值。

我们对程序做如下修改:

In [6]:
# 提取 saying 中与占位符匹配的部分，一同输出
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: 
            return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [7]:
pattern = 'I want ?X'.split()
saying = "I want holiday".split()

In [8]:
pat_match(pattern, saying)

('?X', 'holiday')

In [9]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

# ！提取一组后程序结束

('?X', '2+2')

但是，如果我们的 Pattern 中具备两个变量，那么以上程序就不能解决了，我们可以对程序做如下修改: 


In [10]:
# 提取多个占位符匹配组合
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

于是，我们可以获得： 

In [11]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

如果我们知道了每个变量对应的是什么，那么我们就可以很方便的使用我们定义好的模板进行替换：

为了方便接下来的替换工作，我们新建立两个函数，一个是把我们解析出来的结果变成一个 dictionary，一个是依据这个 dictionary 依照我们的定义的方式进行替换。

In [12]:
# 将提取的匹配结构改为字典;
# 为区别于问句的语法模式 pattrns 改成 pairs 

def pat_to_dict(pairs):
    return {k: v for k, v in pairs}

In [13]:
pat_to_dict([('?X', '3'), ('?Y', '2')])

{'?X': '3', '?Y': '2'}

In [19]:
# 把问句中提取出来的 pair关键词 替换到回答模式中
def substitute(repat, keywords):
    if not repat: return []
    
    return [keywords.get(repat[0], repat[0])] + substitute(repat[1:], keywords)

# The .get() method returns a value for the given key. 
# If key is not available, it returns default value.

In [15]:
got_pairs = pat_match("I want ?X".split(), "I want iPhone".split())

In [16]:
got_pairs

[('?X', 'iPhone')]

In [20]:
substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_pairs))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

为了将以上输出变成一句话，也很简单，我们使用 Python 的 join 方法即可： 

In [23]:
' '.join(substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_pairs)))

'What if you mean if you got a iPhone'

In [24]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())
john_pat

[('?P', 'John'), ('?X', 'vacation')]

In [26]:
substitute("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'vacation', '?']

In [28]:
' '.join(substitute("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

那么如果我们现在定义一些patterns，就可以实现基于模板的对话生成了:

In [29]:
# 定义一些对话模式

defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [35]:
import random

In [49]:
def get_response(saying, repats):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
# 1. Find an entry key in the coversation pattern dict that match the pattern of saying.
# 2. Match and extract keyword pairs.
# 3. If get a pair, substitute variables in an reply pattern with the keywords.
    for k, v in repats.items():
        
        if pat_match(k.split(), saying.split()):
            
            key_pairs = pat_match(k.split(), saying.split())
            pattern = random.choice(v)
            reply = ' '.join(substitute(pattern.split(), pat_to_dict(key_pairs)))
            
            return reply
        
    return "?"
    

In [50]:
get_response('I need iPhone', defined_patterns)

'Why do you need iPhone ?'

In [51]:
get_response("My mother told me something", defined_patterns)

'Talk about more about your mother'

In [52]:
get_response("Hello", defined_patterns)

'?'

### Segment Match

我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？ 

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [54]:
def is_variable_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [55]:
is_variable_segment('?*P')

True

In [56]:
from collections import defaultdict
### ???


然后我们把之前的 ```pat_match```程序改写成如下， 主要是增加了 ``` is_variable_segment ```的部分. 

In [90]:
fail = [True, None]

#提取 saying 中与占位符相匹配的部分，一同输出
# Recursion: Base case -- 1. First word is a variable/ 2. First word is a variable segment/ 
#                         3. First word is a word: match? [y/n] y -- next; n -- fail
#            Operation -- Slice to update the strs as that in pat_match.
#            Stop -- No remaining elements in strs

def pat_match_with_seg(pattern, saying):
    
    if not pattern or not saying: return []
    
    pat = pattern[0]   #'''Word to match'''
    
    if is_variable(pat):  #'''Case 1: the word is a variable'''
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    
    elif is_variable_segment(pat):   #'''Case 2: the word is a variable segment'''
        # 找到 variable segment 对应的句段，返回匹配对和结束字符后一位的位置索引
        
        if segment_match(pattern, saying) != fail:
            
            match, index = segment_match(pattern, saying)
            return [match] + pat_match_with_seg(pattern[1:], saying[index:])
        else: return segment_match(pattern, saying)
    
    elif pat == saying[0]:   #'''Case 3: the word is simply a word'''
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

这段程序里比较重要的一个新函数是 ```segment_match```，这个函数输入是一个以 ```segment_pattern```开头的模式，尽最大可能进行，匹配到这个*边长*的变量对于的部分。

In [91]:
#Preview: All the sentences are split into words and stored in a list. The first word is a variable.

def segment_match(pattern, saying):
    seg, rest = pattern[0], pattern[1:]
    seg = seg.replace('?*', '?')   # str.replace(old, new[, max])
    
    if not rest: return (seg, saying), len(saying)
    
    for i, token in enumerate(saying):
        if rest[0] == token:        # Find the first joint point.
                                    # Leave pat_match_with_seg() to decide whether the rest parts remain all the same.
            return (seg, saying[:i]), i
    
    return fail

In [92]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

现在，我们就可以做到以下的匹配模式了: 

In [93]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

如果我们继续定义一些模板，我们进行匹配，就能够进行更加复杂的问题了: 

In [94]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [149]:
pat_match_with_seg('I was ?*X'.split(), 
                  "I was a good teacher".split())

[('?X', ['a', 'good', 'teacher'])]

In [97]:
substitute("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

 我们会发现，pat_to_dict在这个场景下会有有一点小问题，没关系，修正一些: 

In [126]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

# Python isinstance() The isinstance() function checks if the object (first argument) 
# is an instance or subclass of classinfo class (second argument). 
# The syntax of isinstance() is: isinstance(object, classinfo)

In [99]:
substitute("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

如果我们定义这样的一个模板:

In [100]:
("?*X hello ?*Y", "Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [102]:
substitute("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

### 现在是你的时间了

In [103]:
#我们给大家一些例子: 
    
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

In [140]:
for k, v in rules.items():
    print (k,v)

?*X hello ?*Y ['Hi, how do you do?']
I was ?*X ['Were you really ?X ?', 'I already knew you were ?X .']


### 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [153]:
def get_response(saying, response_rules):

# 1. Find an entry key in the coversation pattern dict that match the pattern of saying.
# 2. Match and extract keyword pairs.
# 3. If get a pair, substitute variables in an reply pattern with the keywords.
    for k, v in response_rules.items():
        
        if pat_match_with_seg(k.split(), saying.split()) != fail:
            
            key_pairs = pat_match_with_seg(k.split(), saying.split())
            pattern = random.choice(v)
            
            reply = ' '.join(substitute(pattern.split(), pat_to_dict(key_pairs)))
            
            return reply
        
    return "?"
    

In [154]:
get_response('I hate you', rules)

'?'

In [135]:

get_response("I am Mike, hello Jenny", rules)

'Hi, how do you do?'

In [155]:
get_response('I was you', rules)

'Were you really you ?'

In [157]:
pat_match_with_seg("I was ?*X".split(), 'I was as tall as J'.split())

[('?X', ['as', 'tall', 'as', 'J'])]

In [159]:
get_response("I was as tall as J", rules)

'I already knew you were as tall as J .'

### 问题2

改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

In [161]:
!pip install jieba

  Using cached https://files.pythonhosted.org/packages/71/46/c6f9179f73b818d5827202ad1c4a94e371a29473b7f043b736b4dab6b8cd/jieba-0.39.zip
  Stored in directory: C:\Users\thinkpad\AppData\Local\pip\Cache\wheels\c9\c7\63\a9ec0322ccc7c365fd51e475942a82395807186e94f0522243
Successfully built jieba


## 增加中文对话功能

In [1]:
import jieba

"""代码示例
# encoding=utf-8

# import jieba

# seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
# print("Full Mode: " + "/ ".join(seg_list))  # 全模式

# seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
# print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

# seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
# print(", ".join(seg_list))

# seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
# print(", ".join(seg_list))"""

import random

In [2]:
# 定义占位符
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

# 定义占位字段
def is_variable_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

# 提取匹配字段
fail = [True, None]
def pat_match_with_seg(pattern, saying):
    
    if not pattern or not saying: return []
    
    pat = pattern[0]   #'''Word to match'''
    
    if is_variable(pat):  #'''Case 1: the word is a variable'''
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    
    elif is_variable_segment(pat):   #'''Case 2: the word is a variable segment'''
        # 找到 variable segment 对应的句段，返回匹配对和结束字符后一位的位置索引
        
        if segment_match(pattern, saying) != fail:
            
            match, index = segment_match(pattern, saying)
            return [match] + pat_match_with_seg(pattern[1:], saying[index:])
        else: return segment_match(pattern, saying)
    
    elif pat == saying[0]:   #'''Case 3: the word is simply a word'''
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail
    
def segment_match(pattern, saying):
    seg, rest = pattern[0], pattern[1:]
    seg = seg.replace('?*', '?')   # str.replace(old, new[, max])
    
    if not rest: return (seg, saying), len(saying)
    
    for i, token in enumerate(saying):
        if rest[0] == token:        # Find the first joint point.
                                    # Leave pat_match_with_seg() to decide 
                                    # whether the rest parts remain all the same.
            return (seg, saying[:i]), i
    
    return fail

# 解析提取的匹配对，设置为字典结构
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

# 替换关键字到回答模式
def substitute(repat, keywords):
    if not repat: return []
    
    return [keywords.get(repat[0], repat[0])] + substitute(repat[1:], keywords)

In [3]:
# 判断该字符是否为中文
def is_Chinese(uchar):

    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
        return True
    else:
        return False

In [13]:
# 获取回应

def get_response(saying, response_rules):
    
    """Mode Detect: check which language the speaker uses"""
    
    if any(is_Chinese(token) for token in saying):
        mode = 'Chinese'
    else:
        mode = 'English'
        
    
    """English Mode"""
   
    if mode == 'English':
        
        for k, v in response_rules.items():
        
            if pat_match_with_seg(k.split(), saying.split()) != fail:
            
                key_pairs = pat_match_with_seg(k.split(), saying.split())
                pattern = random.choice(v)
            
                reply = ' '.join(substitute(pattern.split(), pat_to_dict(key_pairs)))
            
                return reply
        
        return "?"
    
    
    
    """Chinese Mode"""

    if mode == 'Chinese':     

    # A.发起会话的分词 -- list(jieba.cut(saying.replace(' ', '')))
    # B.发起会话匹配模式的分词 -- 定义 seperator
    # C.回应话语匹配模式的分词 -- 使用 seperator
    # D.替换 keywords, 发起最终回应 -- 注意去除空格 ' '

        for k, v in response_rules.items():
        
            saying_cut = list(jieba.cut(saying.replace(' ', ''))) ##切分发起会话

            if pat_match_with_seg(seperator(k), saying_cut) != fail:
                
              
            
                key_pairs = pat_match_with_seg(seperator(k), saying_cut) 
                ## ！key_pairs含有不符合中文习惯的空格
                
                if tuple(fail) in key_pairs:
                    return '?'
                
                pattern = random.choice(v)
                reply = ''.join(substitute(seperator(pattern), pat_to_dict(key_pairs))).replace(' ','')
            
                return reply
        
        return "?"


        

In [5]:
def seperator(input, counter = 0):
    """初始状态默认第一个断句点类型是 en/zh；定义counter统计中英文切换次数，初始值为0
       
       eg: '?*x你好?*y' -- '?*x','你好?*y' #检测到字符切换到中文后，counter = 1
                        -- '?*x','你好','?*y' #检测到字符切换到英文后，counter = 2
    
    """ 
    # WARNING! 符合下例(eg:'?*x喜欢?*y') 的语法习惯,对于以中文开头的句式还有待完善
    # 判断出
    
    for i, token in enumerate(input):
       
        
        if counter%2 == 0:
            
            if is_Chinese(token): #提取连续英文字段，即 variable segment，并切分
                counter += 1
                return input[:i].split() + seperator(input[i:], counter)
        
        else:
            if not is_Chinese(token): #提取连续中文字段，并切分
                counter += 1
                return [x for x in jieba.cut(input[:i])] + seperator(input[i:], counter)
            
    return [input]   

一些参考 pattern

In [6]:
list(jieba.cut('他认识机器人 ANE'))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\thinkpad\AppData\Local\Temp\jieba.cache
Loading model cost 1.052 seconds.
Prefix dict has been built succesfully.


['他', '认识', '机器人', ' ', 'ANE']

In [7]:
seperator('?*x我很难过，因为?*y')

['?*x', '我', '很', '难过', '，', '因为', '?*y']

In [8]:
seperator('你觉得?x和?y有什么相似性？')

['你', '觉得', '?x', '和', '?y', '有', '什么', '相似性', '？']

In [11]:
get_response('他认识机器人 ANE', rule_responses)

[('?x', ['他', '认识']), ('?y', ['ANE'])]


'你为什么要提机器人的事情？'

In [12]:
get_response("AI机器猫就像人一样聪明", rule_responses).replace(' ','')

[('?x', ['AI', '机器猫']), ('?y', ['人', '一样', '聪明'])]


'怎么说？'

In [14]:
get_response('昨天我想不去开会', rule_responses)

'你可以想想你很快就可以不去开会了'

In [15]:
seperator('今天我很难过，因为作业写不完了')

['今天', '我', '很', '难过', '，', '因为作业写不完了']

In [16]:
seperator('?*x我很难过，因为?*y')

['?*x', '我', '很', '难过', '，', '因为', '?*y']

### 问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

In [17]:
get_response('hFOSJFJFSOFKAP', rule_responses)

'我不太确定我很理解你说的, 能稍微详细解释一下吗?'

In [18]:
get_response('Hello, hello, hey', rule_responses)

'请继续'

In [19]:
get_response('Hello hello hey', rule_responses)

'Please state your problem'

### 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 
2. 什么是数据驱动？数据驱动在这个程序里如何体现？
3. 数据驱动与 AI 的关系是什么？ 


In [ ]:
# ？？？ Test failed
# pat_match_with_seg(['?*x', '我', '很', '难过', '，', '因为', '?*y'], '今天我很难过，因为作业写不完了') 
# get_response('今天我很难过，因为作业写不完了', rule_responses)

一些参考 pattern

In [10]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}
